# 로지스틱 회귀: 규제 매개변수 C의 영향 탐색

이 노트북은 `scikit-learn`의 `LogisticRegression` 모델에서 중요한 하이퍼파라미터인 **`C` 매개변수(규제 강도)**가 모델의 복잡성과 성능에 미치는 영향을 탐색합니다.

### `C` 매개변수란?
`C`는 로지스틱 회귀 모델의 **규제(Regularization) 강도**를 제어하는 매개변수입니다. 규제는 모델이 훈련 데이터에 과도하게 적합(과대적합, overfitting)되는 것을 방지하여, 새로운 데이터에 대한 일반화 성능을 향상시키는 데 도움을 줍니다.

- **`C` 값이 작을수록**: 규제가 강해져 모델이 단순해지고, 과소적합(underfitting)될 가능성이 높아집니다.
- **`C` 값이 클수록**: 규제가 약해져 모델이 훈련 데이터에 더 복잡하게 적합하려 하고, 과대적합될 가능성이 높아집니다.

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_breast_cancer # 유방암 데이터셋
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler # 스케일링을 위해 추가

In [ ]:
# 유방암 데이터셋 로드
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

print(f"데이터 형태: {X.shape}")
print(f"타겟 형태: {y.shape}")

### 2. 데이터 분할 및 스케일링

모델의 일반화 성능을 정확히 평가하기 위해 데이터를 훈련 세트와 테스트 세트로 분할하고, `LogisticRegression`은 스케일링에 민감하므로 `StandardScaler`를 사용하여 특성을 표준화합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"훈련 데이터 형태: {X_train_scaled.shape}")
print(f"테스트 데이터 형태: {X_test_scaled.shape}")

### 3. `C` 매개변수 변화에 따른 모델 성능 탐색

다양한 `C` 값에 대해 `LogisticRegression` 모델을 학습시키고, 훈련 세트와 테스트 세트의 정확도를 기록합니다.

In [ ]:
# C 값의 범위 (로그 스케일로 설정)
C_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

train_scores = []
test_scores = []

for C in C_values:
    model = LogisticRegression(C=C, max_iter=5000, random_state=0)
    model.fit(X_train_scaled, y_train)
    train_scores.append(model.score(X_train_scaled, y_train))
    test_scores.append(model.score(X_test_scaled, y_test))
    print(f"C={C: <8.4f} | 훈련 정확도: {train_scores[-1]:.4f} | 테스트 정확도: {test_scores[-1]:.4f}")

### 4. 결과 시각화 및 분석

`C` 값 변화에 따른 훈련 및 테스트 정확도를 그래프로 그려 모델의 복잡성과 성능 간의 관계를 시각적으로 분석합니다.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(C_values, train_scores, label="훈련 세트 정확도")
plt.plot(C_values, test_scores, label="테스트 세트 정확도")
plt.xscale('log') # C 값이 로그 스케일이므로 x축도 로그 스케일로 설정
plt.xlabel("C (규제 강도)")
plt.ylabel("정확도")
plt.title("Logistic Regression: C 값에 따른 훈련/테스트 정확도 변화")
plt.legend()
plt.grid(True)
plt.show()

print("
--- 분석 ---")
print("1. C 값이 매우 작을 때 (예: 0.0001): 규제가 너무 강해 모델이 과소적합되어 훈련 및 테스트 정확도 모두 낮습니다.")
print("2. C 값이 증가함에 따라: 모델의 복잡성이 증가하고 훈련 정확도가 높아집니다. 테스트 정확도도 함께 증가하다가 특정 지점에서 최고점을 찍습니다.")
3. C 값이 너무 클 때 (예: 1000): 규제가 거의 없어 모델이 훈련 데이터에 과대적합되어 훈련 정확도는 매우 높지만, 테스트 정확도는 오히려 감소하거나 정체될 수 있습니다.")
print("
결론적으로, 훈련 정확도와 테스트 정확도 간의 균형을 이루는 적절한 C 값을 찾는 것이 중요합니다.")